### Whisp a GeoJSON via Google drive
- Use this when "whisp_geojson_to_csv' notebook fails due to time out errors
- NB requires collecting outputs from Google Drive 

Import packages
- NB to avoid complications from altering main python environment run using a virtual environment (https://docs.python.org/3/tutorial/venv.html)

In [1]:
import ee

import pandas as pd

from pathlib import Path

import whisp

ee.Authenticate() # Authenticate to Earth Engine (will open a browser window)
ee.Initialize() # Initialize the Earth Engine library


Get a feature collection

In [2]:
GEOJSON_EXAMPLE_FILEPATH = (
    # Path.cwd().parents[0]/ "test" / "fixtures" / "geojson_example.geojson"
    Path.cwd().parents[0]/ "test" / "fixtures" / "mixed_geometries.geojson"
)
print(GEOJSON_EXAMPLE_FILEPATH)


c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\mixed_geometries.geojson


Whisp it

In [6]:
whisp.whisp_stats_geojson_to_drive(GEOJSON_EXAMPLE_FILEPATH)


Received geojson_filepath: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\mixed_geometries.geojson
Reading GeoJSON file from: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\mixed_geometries.geojson
['Plot_area_ha', 'Cocoa_bnetd', 'Oil_palm_Descals', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'Cocoa_ETH', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_FDaP', 'Rubber_FDaP', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_loss_year_2008', 'GFC_loss_year_2009', 'GFC_lo

Manual step 
- download from Google Drive when finished
- place in output folder (or alternative location)



Import CSV of raw stats


In [ ]:
# Define output directory
out_directory = Path.cwd().parents[0] / "test" / "output" # directory untracked by git

# Define output file path
stats_file_from_drive = out_directory / "whisp_output_table.csv" # edit as required

df_stats = pd.read_csv(stats_file_from_drive)


Format stats based on Whisp schema


In [ ]:

df_formatted_stats = whisp.validate_dataframe_using_lookups(df_stats)

Display table

In [ ]:
df_formatted_stats #view output dataframe


,plotId,geoid,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,In_waterbody,...,TMF_def_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,RADD_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020
0,1,NaN,0.089,Point,IDN,Riau,102.0,0.500000,ha,False,...,0.089000,0.08900,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,2,NaN,1230836.875,Polygon,IDN,Riau,100.5,0.500006,ha,False,...,248494.640625,347697.84375,149848.8125,53576.351562,2913.889893,12263.407227,16933.828125,58801.875,1492.604004,32565.113281


Export csv

In [6]:
# Define output directory
out_directory = Path.cwd().parents[0] / "test" / "output" # directory untracked by git

# Define output file path
output_stats_file = out_directory / "whisp_output_table.csv" # edit as required

# Save statistics to CSV
df_formatted_stats.to_csv(path_or_buf=output_stats_file,index=False)

print(f"Formatted stats saved to: {output_stats_file}")


Formatted stats saved to: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\output\whisp_output_table.csv


Calculate risk category

In [7]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = low_name
C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = low_name
C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

Display table with risk columns

In [8]:
df_w_risk

,plotId,geoid,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,In_waterbody,...,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1,NaN,8.939,Point,IDN,Riau,102.0,0.500000,ha,False,...,0.0,0.000000,0.00000,0.000000,0.00000,yes,yes,yes,no,low
1,2,NaN,1231039.750,Polygon,IDN,Riau,100.5,0.500006,ha,False,...,0.0,187.727997,62425.46875,1474.989014,245100.34375,yes,yes,yes,yes,low


Export table with risk columns to csv 

In [9]:
# Define output file path
output_risk_file = out_directory / "whisp_output_table_w_risk.csv" # edit as required

# Save statistics with added risk columns to CSV
df_w_risk.to_csv(path_or_buf=output_risk_file,index=False)

print(f"Table with risk columns saved to: {output_risk_file}")

Table with risk columns saved to: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\output\whisp_output_table_w_risk.csv
